#COMPSCI 546: Applied Information Retrieval
##Assignment 3: Retrieval Models (Total : 100 points)



# Download input files and code

Please execute the cell below to download the input files. 

In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


import zipfile

download = drive.CreateFile({'id': '1obnYvxGG8-x025j2U8aVYf4yBc5mFQsw'})
download.GetContentFile('HW03.zip')

with zipfile.ZipFile('HW03.zip', 'r') as zip_file:
    zip_file.extractall('./')
os.remove('HW03.zip')
# We will use hw1 as our working directory
os.chdir('HW03')

#Setting the input files 
queries_file = "queries_tok_clean_kstem"
col = "antique-collection.tok.clean_kstem"
qrel_file = "test.qrel"
    

# 1 : Initial Data Setup (10 points)

We use files from the ANTIQUE  [https://arxiv.org/pdf/1905.08957.pdf] dataset for this assignment. As described in the previous assignments, this is a passage retrieval dataset. 

The description of the input files provided for this assignment is given below.

**Query File**

We randomly sampled a set of 15 queries from the test set of the ANTIQUE dataset. Each row of the input file contains the following information:

*queryid query_text*

The id and text information is tab separated. queryid is a unique identifier for a query and query text has been pre-processed to remove punctutation, tokenised and stemmed using the Krovetz stemmer.  


**Query Relevance (qrel) file**

The qrel file contains the relevance judgements (ground truth) for the query passage combinations. Each row of the file contains the following information.

*queryid topicid passageid relevance_judgement*

Please note that the entries are space separated. The second column (topicid) can be ignored.

Given below are a couple of rows of a sample qrel file.

*2146313 U0 2146313_0 4*

*2146313 Q0 2146313_23 2*

The relevance judgements range from values 1-4. The description of the labels is given below:

Label 1: Non-Relevant

Label 2: Slightly Relevant

Label 3 : Relevant

Label 4: Highly Relevant

Note: that for metrics with binary relevance assumptions, Labels 1 and 2 are considered non-relevant and Labels 3 and 4 are considered relevant.

Note: if a query-document pair is not listed in the qrels file, we assume that the document is not relevant to the query.


**Collection file**

Each row of the file consists of the following information:

*passage_id  passage_text*

The id and text information is tab separated. The passage text has been pre-processed to remove punctutation, tokenised and stemmed using the Krovetz stemmer (same as queries). The terms in the passage text can be accessed by splitting the text based on space.


In this section, you have to implement the following: 

* Load the queries from the query file into a datastructure
* Load the query relevance information into datastructure. You can reuse some of the code written in Assignment 1 for this and make modifications to it as needed.


You can use any additional datastructures than the suggested ones for your implementation. 




In [ ]:
import pandas as pd
from math import log
from collections import Counter, defaultdict
''' 
This function is used to load query file information into datastructure(s).
Return Variables: 
queries - mapping from queryid to querytext
'''
def loadQueries(queries_file):
    #enter your code here 
    with open(queries_file) as f:  
      qid = []
      qtext = [] 
      qdlist = []
      for line in f:
        temp = line.split("\t")
        qid.append(temp[0])
        qtext.append(temp[1].strip('\n'))
        qlist = temp[1].strip('\n').split()

      queries = pd.DataFrame({'query_id':qid, 'query_txt':qtext})
    
    return queries


'''
This function is used to load qrel file information into datastructure(s).
The qrel file format is the same as the one provided in Assignment 1 and is given below:
"queryid topicid passageid relevance_judgement"
The entries are space separated. 
You can copy your qrel loading code from Assignment 1 and make modifications if necessary.

Return Variables:
num_queries - number of queries in the qrel file
qrels - query relevance information 
'''
def loadQrels(qrel_file):
     #enter your code here 
     qid = []
     pid =[]
     tup = []
     reljudg = []
     qrels = defaultdict(lambda : '')
     with open(qrel_file) as f:   
       for line in f:
         temp = line.split(" ")
         tup = (temp[0], temp[2])
         qrels[tup] = int(temp[3].strip('\n'))

     num_queries = len(qrels)

     return num_queries,qrels

# You can return additional datastructures for your implementation.     
queries = loadQueries(queries_file)
num_queries, qrels = loadQrels(qrel_file)


print ('Total Num of queries in the query file  : {0}'.format(len(queries)))
print ('Total Num of queries in the qrel file  : {0}'.format(num_queries))



Total Num of queries in the query file  : 15
Total Num of queries in the qrel file  : 456



In the cell below, an inverted index with count has been created in memory. Please run the cell and use the variables for implementing the retrieval models. 

In [ ]:
'''
An inverted index with count information. 
'''
class indexCount:
   pcount = 0 
   ctf = {}
   sumdl = 0
   avgdl = 0  
   doclen = {}
   index = {}
   probctf = {}
   
   
   def __init__(self, col):
     self.col = col
     
   
   def create_index(self):
     for line in open(self.col):    
       pid,ptext = line.strip().split('\t')
       self.pcount+=1 
       
       if pid not in self.doclen:
           self.doclen[pid]=0
       pfreq = {}
       for term in ptext.split(' '):
           self.sumdl+=1
           if term not in self.ctf:
             self.ctf[term]=0
           self.ctf[term]+=1 
           self.doclen[pid]+=1 
           
           if term not in pfreq:
              pfreq[term]=0
           pfreq[term]+=1
       
       
       for k,v in pfreq.items():
        if k not in self.index:
          self.index[k]=[]      
        self.index[k].append(pid+":"+str(v)) 

     for k,v in self.ctf.items():
        self.probctf[k]=v/float(self.sumdl) 

     self.avgdl = self.sumdl/float(self.pcount)



     
buildIndex = indexCount(col)      
buildIndex.create_index()


''' 
inverted index with count: dict with term as key and posting list as value
posting list is a list with each element in the format "passage_id:term frequency"
Example - {'the': ['2020338_0:11', '3174498_1:4']} 
'''  
index = buildIndex.index

#total number of passages in the collection 
num_passages = buildIndex.pcount

# Average passage length
avgdl = buildIndex.avgdl

# Collection Term Frequency : dict with term as the key and the term frequency in collection as value
ctf = buildIndex.ctf

# Probability Term Frequencies : dict with terms as key and probability distribution over term frequencies as value
probctf = buildIndex.probctf

#dict with passageId as key and number of tokens in the passage as value  
doclen = buildIndex.doclen

# Total number of tokens in the collection
totNumTerms = buildIndex.sumdl

 
print ('Total number of passages in the collection :{0}'.format(num_passages))
print('Average passage length :{0}'.format(avgdl))
print('Total num of unique terms :{0}'.format(len(ctf)))
print('Total num of terms in the collection :{0}'.format(totNumTerms))


Total number of passages in the collection :403492
Average passage length :41.11619809066846
Total num of unique terms :149467
Total num of terms in the collection :16590057


# 2 : Vector Space model (VSM model) (25 points)

In the cell below, implement the VSM model given in Slide 19 of 'Basic Retrieval Models Part 1'. The score function has been given below for reference.

$$ score(q,p) = \sum_{w \in {q \cap p}} count(w,q) \frac{ln(1+ln(1+count(w,p)))}{1-b+b \frac{|p|}{avgdl}} ln \frac{|C|+1}{df(w)} $$

$score(q,p)$ - score assigned to a passage $p$ for a query $q$

$count(w,q)$ - number of times term $w$ occurs in query $q$

$count(w,p)$ - number of times term $w$ occurs in passage $p$ 

$b$ - set this to 0.75

$|p|$ - Number of tokens in passage $p$

$avgdl$ - Average number of tokens in passages in collection

$|C|$ - number of passages in collection $C$

$df(w)$ - number of passages containing term $w$

Please note that we consider each query term once, since this is equivalent to a dot product. 

For each query, you have to return the top 5 retrieved passages ranked based on the score returned by the VSM model using "term at a time" scoring method. 


In [ ]:
'''
Rank passages for each query and return top 5 passages. 
Return Variables:
final_ranking_vsm : map with query id as key and list of top 5 ranked passages as value                
'''
def vsm(queries, index, avgdl, num_passages, doclen):
   #enter your code here
   b = 0.75
   res_dict = {}
   final_scores = defaultdict(lambda : defaultdict(lambda: 0)) #setting default score to be zero
   final_ranking_vsm = {}

   for ind, row in queries.iterrows():
     qid = row['query_id']
     qterms = Counter(row['query_txt'].split())
     for qt, qtcount in qterms.items():
       dfw = len(index[qt])
       postlist = index[qt]
       hscore = 0
       for pas in postlist:
         passplit = pas.split(":")
         #print(passplit)
         pid = passplit[0]
         countWP = int(passplit[1])
         #score calculation
         ptoken = doclen[passplit[0]]
         t1 = log(1 + log(1 + countWP))
         t2 = 1 - b + ((ptoken * b)/avgdl)
         t3 = log((num_passages + 1)/dfw)
         hscore = (qtcount * (t1/t2) * t3) #for one term in query with passage
         #updating the final score with this
         final_scores[qid][pid] += hscore
      
     final_ranking_vsm[qid] = sorted(final_scores[qid].items(), key=lambda x: x[1], reverse=True)[:5]
   
   return final_ranking_vsm

final_ranking_vsm = vsm(queries, index, avgdl, num_passages, doclen)

# Hint: The score would be in the interval: [13,14]
print ('The top retrieved passage and score for query id "3698636" using VSM is : {0}'.format(final_ranking_vsm['3698636'][:1]))

The top retrieved passage and score for query id "3698636" using VSM is : [('754739_3', 13.046982491248508)]


# 3: BM25 (25 points)

In the cell below, implement the BM25 model given in slide 31 of 'Basic Retrieval Models Part 3'.

$$score(q,p) = \sum_{w \in {q \cap p}} \frac{(k_1+1) count(w,p)}{k_1(1-b+b(\frac{|p|}{avgdl})) + count(w,p)} ln\frac{|C|-df(w)+0.5}{df(w)+0.5}$$


$score(q,p)$ - score assigned to a passage $p$ for a query $q$

$count(w,p)$ - number of times term $w$ occurs in passage $p$ 

$b$ - set this to 0.75

$|p|$ - Number of tokens in passage $p$

$avgdl$ - Average number of tokens in passages in collection

$|C|$ - number of passages in collection $C$

$df(w)$ - number of passages containing term $w$

$k_1$ - set to 1.2 

Please note that we iterate over all query tokens including repetitions. 

Similar to the previous model, return the top 5 retrieved passages for each query ranked based on the BM25 scoring using "term at a time" scoring method. 

In [ ]:
'''
Rank passages for each query and return top 5 passages. 
Return Variables:
final_ranking_bm25 : map with query id as key and list of top 5 ranked passages as value                
'''
def bm25(queries, index, avgdl, num_passages, doclen):
   #enter your code here
   b = 0.75
   k1 = 1.2
   res_dict = {}
   final_scores = defaultdict(lambda : defaultdict(lambda: 0)) #setting default score to be zero
   final_ranking_bm25 = {}

   for ind, row in queries.iterrows():
     qid = row['query_id']
     qterms = Counter(row['query_txt'].split())
     for qt, qtcount in qterms.items():
       dfw = len(index[qt])
       postlist = index[qt]
       hscore = 0
       for pas in postlist:
         passplit = pas.split(":")
         #print(passplit)
         pid = passplit[0]
         countWP = int(passplit[1])
         #score calculation
         ptoken = doclen[passplit[0]]
         num = (k1+1) * countWP
         t1 = 1 - b + ((ptoken * b)/avgdl)
         den = (k1 * t1) + countWP
         t2 = log((num_passages - dfw + 0.5) / (dfw + 0.5))
         hscore = (num/den) * t2  #for one term in query with passage
         #updating the final score with this
         final_scores[qid][pid] += hscore
      
     final_ranking_bm25[qid] = sorted(final_scores[qid].items(), key=lambda x: x[1], reverse=True)[:5]
   

   return final_ranking_bm25


final_ranking_bm25 = bm25(queries, index, avgdl, num_passages, doclen)

# Hint: The score would be in the interval: [18,19]
print ('The top retrieved passage and score for query id "3698636" using BM25 is : {0}'.format(final_ranking_bm25['3698636'][:1]))

The top retrieved passage and score for query id "3698636" using BM25 is : [('3698636_9', 18.355784008150206)]


# 4: Query Likelihood Model (20 points)

In the cell below, implement the Query  Likelihood model using the Jelinek-Mercer (JM) Smoothing technique. The score can be computed as follows:

   
   $$score(q,p) = \sum_{w \in {q}} ln( (1-\lambda) P_{MLE}(w|p) + \lambda P_{MLE}(w|C))$$ 

$score(q,p)$ - score assigned to a passage $p$ for a query $q$

 $P_{MLE}(w|p)$ = $\frac{count(w,p)}{|p|}$ 

 $P_{MLE}(w|C)$ = $\frac{count(w,C)}{N}$

$count(w,p)$ - number of times term $w$ occurs in passage $p$ 

$count(w,C)$ - number of times term $w$ occurs in collection $C$

$|p|$ - Number of tokens in passage $p$


$N$ - Number of tokens in collection $C$

$\lambda$ - set to 0.2

Please note that we iterate over all query tokens including repetitions. 

Similar to the previous model, return the top 5 retrieved passages for each query ranked based on the QL scoring using "term at a time" scoring method. . 

In [ ]:
'''
Rank passages for each query and return top 5 passages. 
Return Variables:
final_ranking_ql_jm: map with query id as key and list of top 5 ranked passages as value           
'''
def ql_jm(queries, index, doclen, probctf):
   #enter your code here
   lmb = 0.2
   final_scores = defaultdict(lambda : defaultdict(lambda: 0)) #setting default score to be zero
   totplist = list(doclen.keys())
   final_ranking_ql_jm = {}

   for ind, row in queries.iterrows():
        qid = row['query_id']
        qterms = Counter(row['query_txt'].split())
        for qt, qtcount in qterms.items():
          postlist = index[qt]
          hscore = 0
          pwc = ctf[qt]/totNumTerms
          tplist = []
          for pas in postlist:
            passplit = pas.split(":")
            pid = passplit[0]
            countWP = int(passplit[1])
            #score calculation
            ptoken = doclen[passplit[0]]
            pwp = countWP/ptoken
            hscore = log(((1 - lmb) * pwp) + (lmb * pwc))
            #updating the final score with this
            final_scores[qid][pid] += hscore
            tplist.append(pid)
          rem_pid = list(set(totplist) - set(tplist))

          for pid in rem_pid:
            final_scores[qid][pid] += log(lmb * pwc)

        final_ranking_ql_jm[qid] = sorted(final_scores[qid].items(), key=lambda x: x[1], reverse=True)[:5]
   

   return final_ranking_ql_jm

final_ranking_ql_jm = ql_jm(queries, index, doclen, probctf)

# Hint: The score value would in the interval [-20,-19]
print ('The top retrieved passage and score for query id "3698636" using Jelinek-Mercer smoothing is : {0}'.format(final_ranking_ql_jm['3698636'][:1]))


The top retrieved passage and score for query id "3698636" using Jelinek-Mercer smoothing is : [('3698636_9', -19.357461855563887)]


# 5: Evaluation (20 points)

In the cell, evaluate the top 5 retrieved passages coresponding to each of the models using Precision@5 and Recall@5 metrics.
You can use the code from assignment 1 modified as needed. 

In [ ]:

# return precision of top 5 retrieved passages
def calcPrecision(top, qrels, rank_in):
     #enter your code here 
     net = 0 
     for key, val in rank_in.items():
       qid = key
       cnt = 0
       for j in val:
         if ((qid, j[0])) in qrels:
           if qrels[(qid, j[0])] >= 3:
             cnt += 1
       prec = (cnt/top)
       net += prec
    
     return net/len(rank_in)

# return recall of top 5 retrieved passages
def calcRecall(top, qrels, rank_in):
    #enter your code here 
    net = 0
    for key, val in rank_in.items():
       qid = key
       cnt = 0
       recos = len(list(k for k, v in qrels.items() if qrels[k] >= 3 and k[0] == qid))
       for j in val:
         if ((qid, j[0])) in qrels:
           if qrels[(qid, j[0])] >= 3:
             cnt += 1
       rec = (cnt/recos)
       net += rec
  
    return net/len(rank_in)   

# Hint: Precision value interval [0.1,0.2],  Recall value interval [0.04,0.05]
print("Evaluate VSM model")
print ('Precision at top 5 : {0}'.format(calcPrecision(5, qrels, final_ranking_vsm)))
print ('Recall at top 5 : {0}'.format(calcRecall(5, qrels, final_ranking_vsm)))
print ("*********************************************************************")
# Hint: Precision value interval [0.3,0.4], Recall value interval [0.10,0.20]
print("Evaluate BM25 model")
print ('Precision at top 5 : {0}'.format(calcPrecision(5, qrels, final_ranking_bm25)))
print ('Recall at top 5 : {0}'.format(calcRecall(5, qrels, final_ranking_bm25)))
print ("*********************************************************************")
# Hint: Precision value interval [0.3,0.4], Recall value interval [0.1,0.2]
print("Evaluate QL model with Jelinek-Mercer smoothing")
print ('Precision at top 5 : {0}'.format(calcPrecision(5, qrels, final_ranking_ql_jm)))
print ('Recall at top 5 : {0}'.format(calcRecall(5, qrels, final_ranking_ql_jm)))
print ("*********************************************************************")



Evaluate VSM model
Precision at top 5 : 0.16
Recall at top 5 : 0.04649035880919939
*********************************************************************
Evaluate BM25 model
Precision at top 5 : 0.36000000000000004
Recall at top 5 : 0.12100747347124159
*********************************************************************
Evaluate QL model with Jelinek-Mercer smoothing
Precision at top 5 : 0.33333333333333337
Recall at top 5 : 0.11713800960177773
*********************************************************************
